In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array


In [ ]:

# Load VGG16 model without top layers
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Ask for training folder path
data_dir = input("Enter path to training images folder: ")

# Load and preprocess images
def load_images(data_dir, image_size=(224, 224)):
    X = []
    y = []
    for file in os.listdir(data_dir):
        if file.lower().endswith(('.jpg', '.jpeg', '.png')):
            label = 0 if 'cat' in file.lower() else 1  # Assumes filenames contain 'cat' or 'dog'
            img_path = os.path.join(data_dir, file)
            try:
                img = load_img(img_path, target_size=image_size)
                img_array = img_to_array(img)
                img_array = preprocess_input(img_array)
                X.append(img_array)
                y.append(label)
            except Exception as e:
                print(f"Error loading {file}: {e}")
    return np.array(X), np.array(y)



In [ ]:
# Load images
X_images, y_labels = load_images(data_dir)

# Extract features using VGG16
features = []
for img_array in X_images:
    img_array = np.expand_dims(img_array, axis=0)
    feature = vgg_model.predict(img_array)
    features.append(feature.flatten())

X = np.array(features)
y = y_labels

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train SVM classifier
svm_model = SVC(kernel='linear', probability=True)
svm_model.fit(X_train, y_train)



In [ ]:
# Evaluate model
y_pred = svm_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# PCA Visualization
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)
plt.figure(figsize=(8, 6))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='coolwarm', edgecolors='k')
plt.title("PCA of VGG16 Features")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.grid(True)
plt.show()


In [ ]:

# Predict a single image manually
def predict_single_image(image_path, model, feature_extractor):
    try:
        img = load_img(image_path, target_size=(224, 224))
        img_array = img_to_array(img)
        img_array = preprocess_input(img_array)
        img_array = np.expand_dims(img_array, axis=0)

        feature = feature_extractor.predict(img_array).flatten().reshape(1, -1)
        prediction = model.predict(feature)[0]
        confidence = model.predict_proba(feature)[0][prediction]

        label = 'Cat' if prediction == 0 else 'Dog'
        print(f"Prediction: {label} ({confidence:.2f} confidence)")
    except Exception as e:
        print(f"Error processing image: {e}")